In [2]:
import torch
from chunkformer_vpb.model_utils import init, load_model_only

# Đường dẫn tới model checkpoint
model_checkpoint = "../../chunkformer-large-vie"

# Chọn device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load model và char_dict
# model, char_dict = init(model_checkpoint, device)
# model.eval()  # Đặt về eval mode

model, char_dict = load_model_only(model_checkpoint, device)

print("✅ Model loaded")
print("📚 Số lượng ký tự trong char_dict:", len(char_dict))



✅ Model loaded
📚 Số lượng ký tự trong char_dict: 6992


In [3]:
# Ví dụ forward dummy input nếu cần
dummy_input = torch.randn(1, 100, 80).to(device)  # batch=1, time=100, mel=80
x_len = torch.tensor([dummy_input.shape[1]], dtype=torch.int32).to(device)

encoder_outs, encoder_lens, _, *_ = model.encoder.forward_parallel_chunk(
    xs=dummy_input,
    xs_origin_lens=x_len,
    chunk_size=64,
    left_context_size=128,
    right_context_size=128,
    att_cache=torch.zeros((model.encoder.num_blocks, 128, model.encoder.attention_heads,
                           model.encoder._output_size * 2 // model.encoder.attention_heads), device=device),
    cnn_cache=torch.zeros((model.encoder.num_blocks, model.encoder._output_size, model.encoder.cnn_module_kernel // 2), device=device),
    truncated_context_size=64 * 4,
    offset=torch.zeros(1, dtype=torch.int, device=device)
)

print("✅ Encoder output shape:", encoder_outs.shape)


✅ Encoder output shape: torch.Size([1, 64, 512])


In [4]:
print(model)


ASRModel(
  (encoder): ChunkFormerEncoder(
    (global_cmvn): GlobalCMVN()
    (embed): DepthwiseConvSubsampling(
      (pos_enc): StreamingRelPositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (out): Linear(in_features=4608, out_features=512, bias=True)
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), groups=512)
        (3): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (4): ReLU()
        (5): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), groups=512)
        (6): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (7): ReLU()
      )
    )
    (after_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (encoders): ModuleList(
      (0-16): 17 x ChunkFormerEncoderLayer(
        (self_attn): StreamingRelPositionMultiHeadedAttention(
          (linear_q): Linear(in_features=512, out_features=